# Imports de bibliotecas necessárias

In [2]:
import json
import pandas as pd
import re
import unicodedata
from datasets import Dataset
from transformers import  Trainer, TrainingArguments, BertTokenizer, BertForQuestionAnswering, pipeline, AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Carregamento e limpeza do arquivo

In [4]:
data = []

# Carregar o arquivo JSON
with open('trn.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Converter para DataFrame
df = pd.DataFrame(data)

# Print da quantidade de linhas antes da limpeza do arquivo
print(f"linhas antes da limpeza:{len(df)}")

# Remove registros inválidos (nulos)
df.dropna(subset=['title', 'content'], inplace=True)

# Remove linhas duplicadas
df.drop_duplicates(subset=['title', 'content'], inplace=True)

# Utiliza clean_text para:
# Remover caracteres especiais
# Normaliza e remove caracteres não ASCII,  afim de facilitar processamento
# Converte para lowercase e remove espaços em branco
def clean_text(text):
    if isinstance(text, str):  # Verificar se é uma string
        text = re.sub(r'[^\w\s]', '', text)
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = text.lower().strip()
    return text

df['title'] = df['title'].apply(clean_text)
df['content'] = df['content'].apply(clean_text)

# Remove registros com conteúdo vazio
df = df[df['content'].str.strip() != '']
df = df[df['title'].str.strip() != '']

# Print da quantidade de linhas após da limpeza do arquivo
print(f"linhas após da limpeza:{len(df)}")

# Path to the output JSON file
file_path = 'data_cleaned.json'

# Grava o df como json para processamento
df.to_json(file_path, orient='records')


linhas antes da limpeza:2248619
linhas após da limpeza:1366477


In [7]:
# Divide as informações de title e content em perguntas e respostas para treinar o modelo
# Realiza a Tokenizacao
# Retorna o dict que contem as questões e contextos tokenizados e as respectivas posicoes de inicio e fim 
def preprocess_function(examples):
    questions = examples['title']  # Perguntas
    contexts = examples['content']  # Respostas
    
    encodings = tokenizer(questions, contexts, truncation=True, padding=True, max_length=512)
    
    start_positions = []
    end_positions = []

    for i in range(len(examples['content'])):
        response = examples['content'][i]
        
        start_idx = response.find(response)
        end_idx = start_idx + len(response) - 1
        
        start_positions.append(start_idx)
        end_positions.append(end_idx)
    
    encodings['start_positions'] = start_positions
    encodings['end_positions'] = end_positions    
    
    return encodings


In [ ]:
from transformers import Trainer, TrainingArguments, BertForQuestionAnswering, BertTokenizer

# Carregar o modelo pré treinado 
# SQuAD (Stanford Question Answering Dataset) 
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Carregar o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Converte o arquivo json para treinamento em dataframe
df_to_train = pd.read_json('data_cleaned.json', orient='records', lines=True)

# Seleciona os primeiros X registros para treinamento
dataset = Dataset.from_pandas(df_to_train.head(1000))

# Realiza a tokenizacao para cada entrada do dataset através da função preprocess_function
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Divide o dataset em treino e teste
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

train_data = split_dataset['train']
eval_data = split_dataset['test']

# Define os parametros de treinamento:
training_args = TrainingArguments(
    output_dir='./results_v2',
    eval_strategy='epoch', 
    learning_rate=3e-5,
    per_device_train_batch_size=4, 
    num_train_epochs=4, 
    weight_decay=0.01,
)

# Inicializa o trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data
)

# Treinar o modelo
trainer.train()

# Salva o modelo treinado
trainer.save_model("./results_v2/qa_fine_tuned_model")


tokenizer.save_pretrained("./results_v2/qa_fine_tuned_model")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Execução modelos

Execução do modelo BERT uncased, sem treinamento com arquivo especializado. 

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")  # Caminho onde você salvou o modelo
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Ou o tokenizer que você usou

context = "high quality 3 layer ballet tutu 12 inches in length"
question = "girls ballet tutu neon pink" 

question_answerer = pipeline("question-answering" , model=model, tokenizer=tokenizer, device=0)

answer  = question_answerer(question=question, context=context)

print(f"Resposta: {answer}")

Execução do modelo bert large (SQUAD - Stanford Question Answering Dataset).

In [ ]:

model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")  # Caminho onde você salvou o modelo
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")  # Ou o tokenizer que você usou

context = "high quality 3 layer ballet tutu 12 inches in length"
question = "girls ballet tutu neon pink" 

question_answerer = pipeline("question-answering" , model=model, tokenizer=tokenizer, device=0)

answer  = question_answerer(question=question, context=context)

print(f"Resposta: {answer}")

Execução do modelo BERT treinado com o arquivo utilizado no trabalho.

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

model = BertForQuestionAnswering.from_pretrained("./results_v2/qa_fine_tuned_model")  # Caminho onde você salvou o modelo
tokenizer = BertTokenizer.from_pretrained("./results_v2/qa_fine_tuned_model")  # Ou o tokenizer que você usou

context = "high quality 3 layer ballet tutu 12 inches in length"
question = "girls ballet tutu neon pink" 

question_answerer = pipeline("question-answering" , model=model, tokenizer=tokenizer, device=0)

answer  = question_answerer(question=question, context=context)

print(f"Resposta: {answer}")